# Step 0. 데이터 준비 개요 (MST 메타데이터 + KIS 시세)

이번 강의에서는 종목 정보를 매번 수동으로 입력하지 않고,  
**MST 마스터 파일(메타데이터)** 과 **KIS 시세 API** 를 함께 사용해서 필요한 정보를 자동으로 가져오겠습니다.

- MST 파일:  
  종목코드, 단축코드, 종목명, 지수/섹터 코드 등 **변하지 않는 메타데이터** (정적 정보)
- KIS API:  
  일별 시세, NAV, 구성종목 등 **시간에 따라 계속 바뀌는 정보** (시계열 데이터)

이 두 가지를 **종목코드 기준으로 미리 정리해 두면**,

- “이 ETF의 섹터/국가/시가총액 분포”
- “상위 보유 종목과 비중”
- “지수 코드/테마 코드와의 매핑”

같은 분석을 할 때, 매번 엑셀/HTS에서 복붙할 필요 없이  
**하나의 공통 테이블(마스터 데이터프레임)** 만 기준으로 삼고 join 해서 사용할 수 있습니다.


## Step 0-1. 공통 환경 설정 & KIS 토큰 유틸 (이전 강의 재사용)

이 셀에서는 이후 모든 예제에서 공통으로 사용할 **기본 환경**을 한 번에 설정합니다.

- `PROJECT_ROOT`, `DATA_DIR`, `RAW_MST_DIR`, `OUT_DIR` 를 만들어  
  MST 파일과 분석 결과를 저장할 **폴더 구조를 통일**합니다.
- `.env` 파일에서 `KIS_URL_BASE`, `KIS_APP_KEY`, `KIS_APP_SECRET`, `KIS_ACCESS_TOKEN` 을 읽어와  
  **KIS OpenAPI 접속 정보**를 불러옵니다.
- `get_api_headers(tr_id)`, `refresh_access_token()`, `validate_and_refresh_token()`, `ensure_kis_token()` 을 정의해서
  - Access Token 발급/갱신
  - 공통 요청 헤더 생성
  을 한 번에 처리하도록 만들어 둡니다.

이 코드는 이전 강의에서 이미 다뤘던 내용을 **공통 유틸 함수로 묶어 놓은 것**이라  
이번 강의에서는 구현 자체를 다시 설명하지 않고,

- 필요할 때 `ensure_kis_token()` 한 줄만 호출해서 토큰을 준비하고,
- 이후 셀에서는 바로 `requests.get(...)` 또는 `get_ohlcv(...)` 같은 래퍼 함수만 사용한다고 생각하면 됩니다.

> 정리하면, 이 셀은  
> “**KIS 환경 설정 + 토큰 관리**를 한 번에 끝내고,  
>  이후 강의에서는 **API 호출 코드 자체가 아니라 데이터 가공·지표 계산에만 집중하겠다**”  
> 라는 준비 단계입니다.


In [ ]:
import os
from pathlib import Path
from typing import Optional

import requests
from dotenv import load_dotenv

PROJECT_ROOT = Path.cwd()

DATA_DIR = PROJECT_ROOT / "data"
DATA_DIR.mkdir(parents=True, exist_ok=True)

load_dotenv(PROJECT_ROOT / ".env")

KIS_URL_BASE: Optional[str] = os.getenv("KIS_URL_BASE")
KIS_APP_KEY: Optional[str] = os.getenv("KIS_APP_KEY")
KIS_APP_SECRET: Optional[str] = os.getenv("KIS_APP_SECRET")
KIS_ACCESS_TOKEN: Optional[str] = os.getenv("KIS_ACCESS_TOKEN")

In [ ]:
def to_float(value) -> Optional[float]:
    try:
        return float(str(value).replace(",", "").strip())
    except Exception:
        return None


"""KIS 공통 헤더 생성 (접근토큰 포함)."""
def get_api_headers(tr_id: str) -> dict[str, str]:
    global KIS_ACCESS_TOKEN

    auth = f"Bearer {KIS_ACCESS_TOKEN}" if KIS_ACCESS_TOKEN else ""

    return {
        "content-type": "application/json; charset=utf-8",
        "authorization": auth,
        "appkey": KIS_APP_KEY or "",
        "appsecret": KIS_APP_SECRET or "",
        "tr_id": tr_id,
        "custtype": "P",  # 개인
    }

In [ ]:
"""KIS OAuth 토큰 재발급 후 메모리 + .env에 반영."""
def refresh_access_token() -> Optional[str]:
    global KIS_ACCESS_TOKEN

    if not KIS_URL_BASE or not KIS_APP_KEY or not KIS_APP_SECRET:
        print("❌ KIS 환경변수(KIS_URL_BASE / KIS_APP_KEY / KIS_APP_SECRET)를 확인하세요.")
        return None

    token_url = f"{KIS_URL_BASE}/oauth2/tokenP"
    headers = {"content-type": "application/json"}
    body = {
        "grant_type": "client_credentials",
        "appkey": KIS_APP_KEY,
        "appsecret": KIS_APP_SECRET,
    }

    try:
        res = requests.post(token_url, headers=headers, json=body, timeout=5)
        data = res.json()
    except Exception as e:
        print(f"❌ 토큰 발급 요청 실패: {e}")
        return None

    access_token = data.get("access_token")
    if not access_token:
        print(f"❌ 토큰 발급 실패: {data}")
        return None

    # 메모리 업데이트
    KIS_ACCESS_TOKEN = access_token
    print("✅ 토큰 발급 성공")

    try:
        env_path = PROJECT_ROOT / ".env"
        lines: list[str] = []
        if env_path.exists():
            lines = env_path.read_text(encoding="utf-8").splitlines()

        new_lines = []
        token_updated = False
        for line in lines:
            if line.startswith("KIS_ACCESS_TOKEN="):
                new_lines.append(f"KIS_ACCESS_TOKEN={access_token}")
                token_updated = True
            else:
                new_lines.append(line)

        if not token_updated:
            new_lines.append(f"KIS_ACCESS_TOKEN={access_token}")

        env_path.write_text("\n".join(new_lines), encoding="utf-8")
        print("   📝 .env 파일 업데이트 완료")
    except Exception as e:
        print(f"   ⚠️ .env 업데이트 실패(무시 가능): {e}")

    return access_token

In [ ]:
def validate_and_refresh_token() -> Optional[str]:
    """
    - 메모리에 토큰이 없으면: 발급
    - 토큰이 있으면: 간단한 테스트 API를 호출해 만료 여부 확인
    - 만료/오류면: 자동 재발급
    """
    global KIS_ACCESS_TOKEN

    if not KIS_URL_BASE:
        raise RuntimeError("KIS_URL_BASE 설정이 없습니다.")

    # 1) 메모리에 토큰이 없으면 바로 재발급
    if not KIS_ACCESS_TOKEN:
        print("⚠️ 메모리에 토큰이 없어 재발급을 시도합니다.")
        return refresh_access_token()

    # 2) 간단한 테스트 API로 토큰 유효성 확인
    test_url = f"{KIS_URL_BASE}/uapi/etfetn/v1/quotations/inquire-price"
    test_params = {
        "FID_COND_MRKT_DIV_CODE": "J",
        "FID_INPUT_ISCD": "069500",  # KODEX200
    }
    test_headers = get_api_headers("FHPST02400000")

    try:
        response = requests.get(test_url, headers=test_headers, params=test_params, timeout=5)
        data = response.json()

        # 토큰 에러 코드 (예: 만료)
        if data.get("rt_cd") in ["EGW00123", "EGW00121"]:
            print("⚠️ 토큰 만료 감지, 재발급 시도...")
            return refresh_access_token()
        elif data.get("rt_cd") == "0":
            print("✅ 기존 토큰 유효함")
            return KIS_ACCESS_TOKEN
        else:
            print(f"⚠️ API 응답 오류 ({data.get('rt_cd')}), 토큰 재발급 시도...")
            return refresh_access_token()

    except requests.exceptions.Timeout:
        print("⚠️ API 응답 시간 초과 (네트워크 이슈일 수 있음, 토큰은 그대로 사용)")
        return KIS_ACCESS_TOKEN
    except Exception as e:
        print(f"⚠️ 토큰 검증 중 오류: {e}")
        return KIS_ACCESS_TOKEN

In [ ]:
def ensure_kis_token() -> Optional[str]:
    """강의 코드에서 '토큰 준비' 용도로 한 줄만 호출하는 함수."""
    token = validate_and_refresh_token()
    if not token:
        print("❌ 토큰 발급/갱신 실패 - KIS 환경변수(.env) 설정을 확인하세요.")
    return token

## Step 0-2. KRX MST 공식 파일 전처리 (주식 마스터만 생성)

이번 강의에서는 ETF 분석에 필요한 **주식 코드/이름 정보만** 사용하므로,
KRX에서 내려받은 MST 파일 중에서

- `kospi_code.mst`
- `kosdaq_code.mst`
- `konex_code.mst`

세 가지만 사용해서 하나의 통합 테이블인  
`mst_fixed/equity_master.csv` 를 만들어 둡니다.

나중에는 이 `equity_master`만 읽어도

- 종목 코드(단축코드, 표준코드)
- 한글 종목명

을 바로 조회할 수 있기 때문에, 분석 코드에서는 더 이상 MST 원본을 직접 다룰 필요가 없습니다.


우선, [한국투자증권 오픈API 개발자센터](https://apiportal.koreainvestment.com) 에 접속하여  
`API문서` -> `종목정보파일` 로 이동하여 필요한 파일들을 다운받아 줍니다.

In [ ]:
from pathlib import Path
import pandas as pd

RAW_MST_DIR = PROJECT_ROOT / "mst_raw"
OUT_DIR     = PROJECT_ROOT / "mst_fixed"

RAW_MST_DIR.mkdir(parents=True, exist_ok=True)
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
def read_mst_lines(path: Path) -> list[bytes]:
    lines = []
    with path.open("rb") as f:
        for raw in f:
            line = raw.rstrip(b"\r\n")  # 레코드 한 줄에서 개행(\r\n) 제거
            if line:                    # 완전히 빈 줄은 스킵
                lines.append(line)
    if not lines:                        # 파일이 비어 있으면 바로 에러
        raise ValueError(f"{path} is empty")
    return lines

# 공통 길이 정의 (MST 파일의 고정폭 필드 길이)
SZ_SHRNCODE  = 9   # 단축코드 길이(보통 6~9자리, KRX가 9바이트로 잡아둔 필드)
SZ_STNDCODE  = 12  # 표준코드(12자리 ISIN/표준코드 계열)
SZ_KORNAME   = 40  # 한글 종목명(40바이트 고정폭)
SZ_KORNAME20 = 20  # 짧은 한글명/회원명 등(20바이트 고정폭)

# 파일별 스키마
EQUITY_SCHEMA = [
    ("short_code", SZ_SHRNCODE),  # 주식/ELW 단축코드 (예: A005930 형태, 9바이트)
    ("std_code",   SZ_STNDCODE),  # 표준코드(종목 표준코드/ISIN 계열, 12바이트)
    ("name",       SZ_KORNAME),   # 종목명(한글, 40바이트)
]

In [ ]:
def parse_fixed_width_lines(lines: list[bytes], schema, *, encoding="cp949") -> pd.DataFrame:
    record_len = max(len(l) for l in lines)
    offsets, offset = [], 0
    for name, length in schema:
        offsets.append((name, offset, length))
        offset += length

    columns = {name: [] for name, _ in schema}
    for line in lines:
        if len(line) < record_len:
            line = line.ljust(record_len, b" ")
        for name, start, length in offsets:
            raw = line[start:start+length]
            columns[name].append(raw.decode(encoding, errors="ignore").rstrip())
    return pd.DataFrame(columns)

In [ ]:
def build_equity_master() -> pd.DataFrame:
    dfs = []
    for fname in ("kospi_code.mst", "kosdaq_code.mst", "konex_code.mst"):
        path = RAW_MST_DIR / fname
        if not path.exists():
            print(f"⚠️ {path} 없음, 스킵")
            continue
        lines = read_mst_lines(path)
        df = parse_fixed_width_lines(lines, EQUITY_SCHEMA)
        dfs.append(df)

    if not dfs:
        raise RuntimeError("MST 원본을 찾지 못했습니다.")

    merged = pd.concat(dfs, ignore_index=True)
    out_csv = OUT_DIR / "equity_master.csv"
    merged.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"[SAVE] equity_master.csv → {out_csv}")
    return merged

In [ ]:
df = build_equity_master()
print(df.head())

## Step 0-3. `kis` 모듈로 ETF 이름 조회 테스트

방금 `mst_fixed/equity_master.csv` 를 만들어 두었으니,
이제는 이 파일을 직접 다루지 않고,

- `load_equity_master()`
- `get_etf_name(etf_code)`

같은 **재사용 가능한 유틸 함수**를 통해서

- 종목 이름으로 코드 후보 찾기
- ETF 코드(6자리)로 한글 이름 조회하기

동작을 간단히 확인해 보겠습니다.

실무/다음 강의에서는 이 모듈들을 그대로 가져와서
- ETF 기본 정보 표기
- 구성 종목 이름/비중 표시
등에 재사용하게 됩니다.

In [ ]:
def load_equity_master() -> pd.DataFrame:
    eq_path_parquet = OUT_DIR / "equity_master.parquet"
    eq_path_csv = OUT_DIR / "equity_master.csv"

    if eq_path_parquet.exists():
        df = pd.read_parquet(eq_path_parquet)
    elif eq_path_csv.exists():
        df = pd.read_csv(eq_path_csv)
    else:
        raise FileNotFoundError(
            f"equity_master 파일을 찾을 수 없습니다: {eq_path_parquet} / {eq_path_csv}"
        )

    for col in ("name", "short_code", "std_code"):
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip()

    return df

In [ ]:
def _extract_6digit_code(s: str) -> Optional[str]:
    if not isinstance(s, str):
        return None
    digits = "".join(ch for ch in s if ch.isdigit())
    if len(digits) < 6:
        return None
    return digits[-6:]

In [ ]:
def get_etf_name_from_mst(etf_code: str) -> Optional[str]:
    df = load_equity_master()

    if "kis_code" not in df.columns:
        if "short_code" in df.columns:
            df["kis_code"] = df["short_code"].apply(_extract_6digit_code)
        elif "std_code" in df.columns:
            df["kis_code"] = df["std_code"].apply(_extract_6digit_code)

    if "kis_code" not in df.columns:
        return None

    m = df["kis_code"] == str(etf_code)
    hit = df[m]
    if hit.empty:
        return None

    name = str(hit.iloc[0]["name"]).strip()
    return name or None

In [ ]:
def find_code_by_name(kor_name: str) -> pd.DataFrame:
    df = load_equity_master()
    m = df["name"].str.contains(kor_name, case=False, na=False)
    candidates = df[m].copy()

    if candidates.empty:
        print(f"⚠️ 이름에 '{kor_name}' 이(가) 포함된 종목을 찾지 못했습니다.")
        return candidates

    if "kis_code" not in candidates.columns:
        if "short_code" in candidates.columns:
            candidates["kis_code"] = candidates["short_code"].apply(_extract_6digit_code)
        elif "std_code" in candidates.columns:
            candidates["kis_code"] = candidates["std_code"].apply(_extract_6digit_code)
        else:
            candidates["kis_code"] = None

    return candidates

In [ ]:
def pick_single_code(kor_name: str) -> str:
    candidates = find_code_by_name(kor_name)
    if candidates.empty:
        raise ValueError(f"'{kor_name}' 에 해당하는 종목을 찾을 수 없습니다.")

    row = candidates.iloc[0]
    kis_code = row.get("kis_code")
    if not kis_code:
        raise ValueError(
            f"'{kor_name}' 후보에서 KIS 코드(6자리)를 추출하지 못했습니다.\n{row}"
        )

    print(f"[INFO] '{kor_name}' → 선택된 종목: {row.get('name')} (kis_code={kis_code})")
    return str(kis_code)

## Step 0-4. ETF 코드 → 한글 이름 조회 (MST + KIS API)

마지막으로, ETF 6자리 코드를 넣으면

1. 먼저 MST 마스터(`equity_master`)에서 이름을 찾아보고,
2. 거기에 없으면 KIS OpenAPI를 한 번 조회해서 이름을 보완하는

`get_etf_name(etf_code)` 함수를 만들어 보겠습니다.

이 함수는 뒤 강의(구성종목 시세 조회, 뉴스 분석 등)에서
그래프/표에 ETF 이름을 깔끔하게 표시할 때 계속 재사용하게 됩니다.

In [ ]:
def get_etf_name_from_mst(etf_code: str) -> Optional[str]:
    df = load_equity_master()

    if "kis_code" not in df.columns:
        if "short_code" in df.columns:
            df["kis_code"] = df["short_code"].apply(_extract_6digit_code)
        elif "std_code" in df.columns:
            df["kis_code"] = df["std_code"].apply(_extract_6digit_code)

    if "kis_code" not in df.columns:
        return None

    m = df["kis_code"] == str(etf_code)
    hit = df[m]
    if hit.empty:
        return None

    name = str(hit.iloc[0]["name"]).strip()
    return name or None

In [ ]:
def get_etf_name_from_kis(etf_code: str) -> Optional[str]:
    validate_and_refresh_token()
    if not KIS_URL_BASE:
        return None

    url = f"{KIS_URL_BASE}/uapi/domestic-stock/v1/quotations/search-stock-info"
    params = {"PRDT_TYPE_CD": "512", "PDNO": etf_code}
    headers = get_api_headers("CTPF1604R")

    try:
        res = requests.get(url, headers=headers, params=params, timeout=5)
        data = res.json()
    except Exception as e:
        print(f"⚠️ KIS ETF 이름 조회 실패: {e}")
        return None

    if data.get("rt_cd") != "0" or not data.get("output"):
        return None

    output = data["output"]
    row = output[0] if isinstance(output, list) else output
    name = row.get("prdt_name") or row.get("hts_kor_isnm")
    name = (name or "").strip()
    return name or None

In [ ]:
def get_etf_name(etf_code: str) -> str:
    code = str(etf_code)

    name = get_etf_name_from_mst(code)
    if name:
        print(f"✅ ETF 이름(MST): {code} → {name}")
        return name

    name = get_etf_name_from_kis(code)
    if name:
        print(f"✅ ETF 이름(KIS): {code} → {name}")
        return name

    print(f"⚠️ ETF 이름 조회 실패 → 코드 사용 ({code})")
    return code

In [ ]:
# Step 0 전체 동작 테스트

# 1) equity_master 로딩
eq = load_equity_master()
print("equity_master shape:", eq.shape)
display(eq.head())

# 2) 코드 → ETF 이름 (MST 우선, 필요시 KIS)
ensure_kis_token()
etf_name = get_etf_name(kodex200_code)
print("\n최종 ETF 이름:", etf_name)